In [ ]:
import pandas as pd
import ppscore as pps
from ruff_model import ruff_model, my_predictors
import seaborn as sns
import numpy as np

In [ ]:
import re

In [ ]:
from IPython.display import display

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%load_ext autoreload
%autoreload 2

# "Will it snow tomorrow?" - The time traveler asked
The following dataset contains climate information form over 9000 stations accross the world. The overall goal of t"hese subtasks will be to predict whether it will snow tomorrow 13 years ago. So if today is 2022.02.15 then the weather we want to forecast is for the date 2009.02.16. You are suppsed to solve the tasks using Big Query, which can be used in the Jupyter Notebook like it is shown in the following cell. For further information and how to used BigQuery in Jupyter Notebook refer to the Google Docs. 

The goal of this test is, to test your coding knowledge in Python, BigQuery and Pandas as well as your understanding of Data Science. If you get stuck at the first part, you can use the replacement data provided in the second part

In [ ]:
import os
os.environ["GCLOUD_PROJECT"] = "learn-360918"

In [ ]:
%%bigquery 
SELECT
*,
FROM `bigquery-public-data.samples.gsod`
LIMIT 20


## Part 1

### 1. Task
Change the date format to 'YYYY-MM-DD' and select the data from 2005 till 2009 for station numbers including and between 725300 and 726300 , and save it as a pandas dataframe. Note the maximum year available is 2010. 

In [ ]:
%%bigquery df
SELECT
DATE( year, month, day ) AS date,
*,
FROM `bigquery-public-data.samples.gsod`
WHERE (station_number BETWEEN 725300 AND 726300)
AND (year BETWEEN 2005 AND 2009)
LIMIT 10 # should in principle be removed. left to avoid downloading to much stuff.

### 2. Task 
From here want to work with the data from all stations 725300 to 725330 that have information from 2005 till 2009.

In [ ]:
%%bigquery raw_data
SELECT
DATE( year, month, day ) AS date,
*,
FROM `bigquery-public-data.samples.gsod`
WHERE (station_number BETWEEN 725300 AND 725330)
AND (year BETWEEN 2005 AND 2009)

Which of the stations 725300 to 725330 have information from 2005 till 2009?

In [ ]:
raw_data = raw_data.assign(date=lambda df: pd.to_datetime(df.date))
raw_data.groupby('station_number').agg({'date':["min", "max", 'size']})

Do a first analysis of the remaining dataset, clean or drop data depending on how you see appropriate. 

### Droping columns and data imputation

In [ ]:
(raw_data.drop(columns=['year', 'month', 'day', 'date', 'wban_number'])
 .pipe(lambda df: df.drop( columns=[col for col in df.columns if col.endswith('_samples')] ))
 .groupby('station_number').agg(lambda x:(x.isna()).mean()).astype('float'))

Since we are doing a very ruff analysis we will just drop the "_samples" columns and all columns missing more than 10% of their values. 
Normally we should take a bit to think if there are to few "_samples" for something and if we can use more of the columns. 
Additionally we impute zero for missing snow_depth values.

In [ ]:
stations = sorted(set(raw_data['station_number']))
renumbering = dict(zip(stations,range(1,len(stations)+1)))

cleaned_df = (raw_data
              .assign(snow_depth=lambda df: df.snow_depth.fillna(0),
                     station_number=lambda df:df.station_number.replace(renumbering))
   [lambda df:[col for col in df.columns 
               if not col.endswith('_samples') 
                   and not df[col].isna().mean() > 0.1 
                   and col not in {'year','month','day','wban_number','max_temperature_explicit'}]])

Next we impute the rest of the missing values by linear interpolation over time. 
In practice we should also consider droping data, filling with values from stations near by and doing other types of interpolation.

In [ ]:
(cleaned_df.groupby('station_number').agg(lambda x:(x.isna()).mean()).astype('float'))

In [ ]:
cleaned_df = cleaned_df.sort_values(['station_number', 'date']).pipe(lambda df: df.assign(**{col:df[col].interpolate() for col in df.columns if df[col].dtype.kind == 'f'})).sort_values(['date'])

In [ ]:
(cleaned_df.groupby('station_number').agg(lambda x:(x.isna()).mean()).astype('float'))

# Doing a first analysis

In [ ]:
(~cleaned_df.snow).mean()

In [ ]:
def fahrenheit_to_celsius(f):
    return (f - 32) * 5/9

def knots_to_kmph(k):
    return 1.852*k

def inches_to_cm(in_):
    return 2.54*in_

def miles_to_km(mi):
    return 1.60934*mi

def convert_units(df):
    return df.assign(month=lambda df:df.date.dt.month,
            mean_temp=lambda df:fahrenheit_to_celsius(df.mean_temp),
            max_temperature=lambda df:fahrenheit_to_celsius(df.max_temperature),
            mean_dew_point=lambda df:fahrenheit_to_celsius(df.mean_dew_point),
            mean_visibility=lambda df:miles_to_km(df.mean_visibility),
            mean_wind_speed=lambda df:knots_to_kmph(df.mean_wind_speed),
            max_sustained_wind_speed=lambda df:knots_to_kmph(df.max_sustained_wind_speed),
            total_precipitation=lambda df:inches_to_cm(df.total_precipitation),
            snow_depth=lambda df:inches_to_cm(df.snow_depth),
           )

In [ ]:
(cleaned_df
 .pipe(convert_units)
 .groupby('month').mean()
 .round(3)
 )

Data corrupted? fog,rain,snow,hail,thunder **all the same**

#### Compare with known data to get a feeling for it, would also be interesting to reproduce summary data if that was avilable.

In [ ]:
(pd.read_html('https://en.wikipedia.org/wiki/Berlin',match='Climate data for Berlin \(Schönefeld\)')[0]
    .apply(lambda x: x.str.replace(r"\([^\(\)]*\)",""))
 .T
 .droplevel(level=0)
 .pipe(lambda df: df.rename(columns=df.iloc[0].str.replace(r'\s*°C\s*','')))
 .pipe(lambda df: df.drop(df.index[[0]]))
 .assign(month=list(range(1,13))+['year'])
 .set_index('month')
 [['Record high','Average high','Daily mean']]
)

In [ ]:
(cleaned_df
 [lambda df: df.station_number == 1]
 .pipe(convert_units)
 .groupby(['month'])
 .agg({'max_temperature':['max','mean'], 'mean_temp':'mean'})
 .apply(lambda x: round(x, 2))
 )

In [ ]:
(cleaned_df
 .pipe(convert_units)
 .groupby(['station_number'])
 .agg({'max_temperature':['max','mean'], 'mean_temp':'mean'})
 .apply(lambda x: round(x, 2))
 )

### We prefer predictive power score over correlation

In [ ]:
pps.predictors(cleaned_df.sample(10 ** 4), 'snow', sample=None)

In [ ]:
(~cleaned_df.snow).mean()

Notice that the pps model_score value is not comparable with the accuacy of a naive model predicting always snow since it is an F1 score!

### The imbalance in the data could be a problem:

In [ ]:
def balance(df, col):
    return pd.concat([df[lambda df: df[col]],
                         df[lambda df: ~df[col]].sample(df[col].sum())])
balanced_df = balance(cleaned_df, 'snow')
balanced_df.snow.mean()

In [ ]:
pps.predictors(balanced_df, 'snow', sample=None)

In [ ]:
matrix_df =pps.matrix(balanced_df.drop(columns=['fog', 'rain', 'hail', 'thunder', 'tornado']))[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore').round(2)
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)

In [ ]:
matrix_df = (pps.matrix(balanced_df[['snow', 'date', 'mean_temp', 'max_temperature', 'station_number', 'total_precipitation']])
             [['x', 'y', 'ppscore']]
             .pivot(columns='x', index='y', values='ppscore')
             .round(2))
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)

In [ ]:
cleaned_df = cleaned_df[['snow', 'date', 'max_temperature', 'station_number', 'total_precipitation']]

## Part 2
If you made it up to here all by yourself, you can use your prepared dataset to train an Algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:

In [ ]:
import datetime as dt

str(dt.datetime.today()- dt.timedelta(days=13*365)).split(' ')[0]

You are allowed to use any library you are comfortable with such as sklearn, tensorflow keras etc. 
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1. 

# Model Architecture
Again due to time constraints we decided to model the data as follows:
 - balance the training data
 - put together fixed length time series into data vectors
 - limited training data for each station prediction --> use one model predicting just snow or no snow from the data of the stations most suited for it
 - train a rbf-kernel SVC on that

In [ ]:
def join_multi_index(df):
    df.columns = ["_".join(map(str,col)) for col in df.columns.values]
    return df

time_data = (pd.concat([(cleaned_df
            .assign(date=lambda df:df.date+i*pd.Timedelta('1d'),
                  date_shift=i
                  )
           ) for i in range(0,5)])
 .pivot(index='date', values=['max_temperature','total_precipitation', 'snow'],columns=['station_number','date_shift'])
 .pipe(join_multi_index)
 .dropna()
 .reset_index()
 .convert_dtypes()
)
time_data

In [ ]:
pps.predictors(balance(time_data,'snow_1_0'), 'snow_1_0', sample=None)[lambda df: ~df.x.str.endswith('0')]

In [ ]:
def score(i,j):
    # how good does snow_i_0 predict snow_j_0
    return pps.score(balance(time_data, f'snow_{j}_0'), f'snow_{i}_0', f'snow_{j}_0')['ppscore']

matrix = np.array([[score(i,j) for j in range(1,11)] for i in range(1,11)])
matrix_df = pd.DataFrame(matrix, index=pd.Series(range(1,11),name='y'), columns=pd.Series(range(1,11),name='x')).round(2)
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)

In [ ]:
best_stations = [[i]+sorted(set(range(1,11))-{i,8}, key = lambda x:score(i,x), reverse=True)[:2] for i in range(1,11) if i != 8]
best_stations

In [ ]:
stations = best_stations[2]

def new_names(stations):
    return ({f"snow_{stations[0]}_0":'_y'}
            |{"date":"_date"}
            |{f"{col}_{start}_{time}":f"_{col}_{target}_{time}" 
                 for col in ['max_temperature','total_precipitation','snow']
                 for target, start in tuple(enumerate(stations,1))
                 for time in range(1,5)
             })

In [ ]:
train_df = pd.concat([(time_data
 .rename(columns=new_names(stations))
 .pipe(lambda df: df.drop(columns=[col for col in df.columns if col[0] != '_']))
 .rename(columns=lambda x:x.lstrip('_'))
) for stations in best_stations])

In [ ]:
groups = set(map(lambda x:x.strip('_'+"".join(map(str,range(10)))), train_df.columns))-{'y'}

In [ ]:
col_groups = [sorted([col for col in train_df.columns if col.startswith(group)]) for group in groups]

In [ ]:
ruff_model(train_df,col_groups,['y'], n=100)